## In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame.

## 1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.  

In [3]:
import requests

In [4]:
URL = 'https://realpython.github.io/fake-jobs/'

response = requests.get(URL)

In [5]:
from bs4 import BeautifulSoup as BS

In [6]:
soup = BS(response.text)

## a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  

In [8]:
soup.find('h2').text

'Senior Python Developer'

## b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.  

In [10]:
titles = soup.findAll('h2')

In [11]:
titles = [x.text for x in titles]

## c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.  

In [13]:
companies = soup.findAll('h3')

In [14]:
companies = [x.text for x in companies]

In [15]:
locations = soup.findAll('p', attrs={'class' : 'location'})

In [16]:
#need to clean this one -- remove the \n things, and the white spaces leading and trailing
locations = [x.text.replace('\n','').strip() for x in locations]

In [17]:
postdate = soup.findAll('time')

In [18]:
postdate = [x.text for x in postdate]

## d. Take the lists that you have created and combine them into a pandas DataFrame. 

In [20]:
import pandas as pd

In [21]:
results_df = pd.DataFrame()

In [22]:
results_df['title'] = titles
results_df['company'] = companies
results_df['location'] = locations
results_df['date'] = postdate

In [23]:
results_df

,title,company,location,date
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


## 2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.

## a. First, use the BeautifulSoup find_all method to extract the urls.  

In [26]:
applyurls = soup.findAll('a')

In [27]:
applyurls = [x.get('href') for x in applyurls if x.text == 'Apply']

In [28]:
results_df['application'] = applyurls

## b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [30]:
# https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html',

# loop through titles, lowercase, remove ( and ) and ',', replace spaces with -, and add -x to the back before html.
# append it back to the dataframe, and run a check to make sure they are equal
newurls = titles.copy()

In [31]:
replacements = {'(': '',
                ')': '',
                ',': '',
                '/': ' ',         # / should become a space
                ' ': '-'}

In [32]:
for i in range(len(newurls)) :
    newurls[i] = newurls[i].lower()
    for old, new in replacements.items() :
        newurls[i] = newurls[i].replace(old, new)
    newurls[i] = 'https://realpython.github.io/fake-jobs/jobs/' + newurls[i] + '-' + str(i) + '.html'

In [33]:
applyurls == newurls # are they the same?  At least for the set that I'm testing against?

True

## 3. Finally, we want to get the job description text for each job.  

## a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.  

In [36]:
joburl = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'

In [37]:
response = requests.get(joburl)

In [38]:
jobsoup = BS(response.text)

In [39]:
description = jobsoup.find('div', attrs={'class' : 'content'}).find('p').text

In [40]:
description

'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.'

## b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".  

In [42]:
def get_desc(url) :
    response = requests.get(url)
    thesoup = BS(response.text)
    return thesoup.find('div', attrs={'class' : 'content'}).find('p').text

In [43]:
get_desc("https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html")

'At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.'

## c. Use the [.apply method](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) on the url column you created above to retrieve the description text for all of the jobs.

In [45]:
results_df['desc'] = ''

for index, row in results_df.iterrows() :
    results_df.loc[index, 'desc'] = get_desc(row['application'])

In [46]:
results_df

,title,company,location,date,application,desc
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...,Professional asset web application environment...
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...,Party prevent live. Quickly candidate change a...
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/le...,Administration even relate head color. Staff b...
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fi...,Tv program actually race tonight themselves tr...
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/pr...,Traditional page a although for study anyone. ...
...,...,...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/mu...,Paper age physical current note. There reality...
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/ra...,Able such right culture. Wrong pick structure ...
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/da...,Create day party decade high clear. Past trade...
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fu...,Pressure under rock next week. Recognize so re...


In [48]:
job_listings['desc2'] = get_desc['application'].apply(job_desc)

AttributeError: 'function' object has no attribute 'application'